# Naive Bayes Text Classifier

A Naive Bayes classifier is a supervised machine learning algorithm that leverages Bayes' Theorem to make predictions and classifications.

Bayes theorem:

`P(A|B)` = (`P(B|A)` * `P(A)`) / `P(B)`

finds the probability that `A` will occur given that `B` is true/has occured.

This can be turned into a classifier if we replace `B` with a data point and `A` with a class.

Naive Bayes classifiers are often used for text classification. So if we're trying to classify email as spam or not spam, we could calculate `P(spam | email)` and `P(not spam | email)`.  Whichever probability is higher will be the classifier's prediction.

In order to compute the probabilities used in Bayes' theorem, we need previous data points. For example, in the spam example, we'll need to compute `P(spam)`. This can be found by looking at a tagged dataset of emails and finding the ratio of spam to non-spam emails.

## Create Naive Bayes Text Classifier

That can predict whether a review for a product is positive or negative. This type of classifier would be helpful for a company that is curious about the public reaction to a new product. Rather than reading thousands of reviews or tweets about the product, you could feed those documents into the Naive Bayes classifier and instantly find out how many are positive and how many are negative.

We’re going to write a classifier that can predict whether the review "This crib was amazing" is a positive or negative review.

The only features of the dataset that we’re interested in are the text of the review and whether the review was "positive" or "negative". All reviews with a score less than 4 are considered negative.

We want to compute both `P(positive | review)` and `P(negative | review)` and find which probability is larger. To do this, we’ll be using Bayes' Theorem:

`P(positive | review)` = (`P(review | positive)` * `P(positive)`) / `P(review)`

`P(positive | review)` --> given a review, probability that it is positive

`P(review | positive)` --> given review is positive, calculate the probability that the words "This", "crib", "was", and "amazing" are the only words in the review?

`P(positive)` --> the probability that any review is positive. Look at all of our reviews in our dataset - both positive and negative - and find the percentage of reviews that are positive.

`P(review)` --> is the probability that "This", "crib", "was", and "amazing" are the only words in the review, ignores the whether the review is positive or negative.

#### First - `P(positive)` - calculate the probability that any review is positive 

```py
# total number of positive reviews - length of pos_list
# total number of negative reviews - length of neg_list
total_reviews = len(pos_list) + len(neg_list)
```

```py
percent_pos = len(pos_list) / total_reviews # 0.5
percent_neg = len(neg_list) / total_reviews #0.5
```

#### Second - calculate  `P(review | positive)`

Given that  the review is positive, what is the probability that the words "This", "crib", "was", and "amazing" are the only words in the review?

We have to assume that each word is conditionally independent. This means that one word appearing doesn't affect the probability of another word from showing up(big assumption).

This gives us the following equation:

`P(“This crib was amazing" ∣ positive)` = `P(“This" ∣ positive)` * `P(“crib" ∣ positive)`  * `P(“was" ∣ positive)`  * `P(“amazing" ∣ positive)`

If we we're to examine `P(“crib" ∣ positive)`, is the probability that the word "crib" appears in a positive review. To find this, we need to count up the total number of times "crib" appeared in our dataset of positive reviews. If we take that number and divide it by the total number of words in our positive review dataset, we will end up with the probability of "crib" appearing in a positive review.

`P(“crib" ∣ positive)` = `num times "crib" in positive` / `num of words in positive`

Repeat for `P(“This" ∣ positive)`, `P(“was" ∣ positive)` and `P(“amazing" ∣ positive)`.

```py
# total number of words in all positive & negativereviews
# note: pos_counter & neg_counter are dictionaries
total_pos = sum(pos_counter.values())
total_neg = sum(neg_counter.values())
```

```py
pos_probability = 1.0
neg_probability = 1.0
review = "This crib was amazing"
review_words = review.split(' ')

# calculate the probility of each word in turn appearing in both positive and negative reviews
for word in review_words:
  word_in_pos = pos_counter[word]
  word_in_neg = neg_counter[word]
  
  pos_probability *= word_in_pos / total_pos
  neg_probability *= word_in_neg / total_neg
  
print(pos_probability) # 7.684476462488163e-13
print(neg_probability) # 2.389642284511267e-13  
```

The `pos_probability` and `neg_probability` are `P("This crib was amazing"|positive)` and `P("This crib was amazing"|negative)` respectively.

#### Third - Smooth the data

It's possible that the word 'crib' is not found in the data set resulting in `P(“crib" ∣ positive)` being 0. This would also result in `P(“This crib was amazing" ∣ positive)` being 0, since the individual probabilities are multiplied together.

This is especially problematic if there are typos in the review we are trying to classify. If the unclassified review has a typo in it, it is very unlikely that that same exact typo will be in the dataset, and the entire probability will be 0. 

If we change the review to `This cribb was amazing`, the pos/neg_probability are 0.

To solve this problem, we use a technique called `smoothing`.

`Smoothing` involves adding 1 to the numerator of each probability and N to the denominator of each probability. N is the number of unique words in our review dataset.

Thus:

`P('crib' ∣ positive)` = `num times 'crib' in positive` / `num of words in positive`

becomes:

`P('crib' ∣ positive)` = `num times 'crib' in positive` + 1 / `num of words in positive` + N

```py
# updated pos/neg_probability calculation
for word in review_words:
  word_in_pos = pos_counter[word]
  word_in_neg = neg_counter[word]
  
  pos_probability *= (word_in_pos + 1) / (total_pos + len(pos_counter))
  neg_probability *= (word_in_neg + 1) / (total_neg + len(neg_counter))
  
print(pos_probability) # 3.3305791603855448e-16
print(neg_probability) # 1.245211661647697e-16
```

#### Fourth -  `P(review)`

`P(review)` is the probability that "This", "crib", "was", and "amazing" are the only words in the review. Notice that this is similar to P(review | positive). The only difference is that we don’t assume that the review is positive.

However, we want to predict whether the review "This crib was amazing" is a positive or negative review. In other words, we’re asking whether P(positive | review) is greater than P(negative | review). This can be calculated with the following two equations:

`P(positive | review)` = (`P(review | positive)` * `P(positive)`) / `P(review)`

and

`P(negative | review)` = (`P(review | negative)` * `P(negative)`) / `P(review)`

Since `P(review)` is the denominator in both cases, we can ignore it. We ignore the denominator of Bayes Theorem when calculating the probabilities of each class

```py
final_pos = pos_probability * percent_pos
final_neg = neg_probability * percent_neg
  
print(final_pos)
print(final_neg)
  
if final_pos > final_neg:
  print('The review is positive')
else:
  print('The review is negative')

# review = "This crib was amazing" --> 'The review is positive'
# review = "This crib was awful"   --> 'The review is negative'
```

The text xlassifier is complete. If you have a dataset of text that has been tagged with different classes, you can give your classifier a brand new document and it will predict what class it belongs to.